In [9]:
import sys
sys.path.append('/Users/sanjaydevarajan/Desktop/ML Projects/PremierLeague_TicketDemand_Predictor')



In [10]:
from src.components.data_info import *
from src.paths import *
import pandas as pd

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def plot_demand_trend(data,start_date, end_date, region_code):
    """
    Plots the demand trend for a specific regioon  within a given date range.

    Parameters:
        data(pd.DataFrame) : The dataframe containing the demand data.
        start_date (str): The start date for the plot in the 'YYYY-MM-DD' format.
        end_date (str): The end date for the plot in 'YYYY-MM-DD' format.
        region_code (int): The sub_region_cide to filter the data for plotting.
    """

    if data['date'].dtype == object:
        data['date'] = pd.to_datetime(data['date'])


    # Filter data based on date range and region
    mask = (data['date'] >= start_date) & (data['date'] <= end_date) & (data['sub_region_code'] == region_code)
    filtered_data = data[mask]

    if filtered_data.empty:
        print(f"No data available for Region {region_code} between {start_date} and {end_date}")
        return
    
    # Plotting
    plt.figure(figsize=(10,8))
    sns.lineplot(data=filtered_data, x="date", y="demand", label="sub_region_code")
    plt.title(f"Demand Trend for Region {region_code} from {start_date} to {end_date}")

    plt.xlabel("Date")
    plt.ylabel("Demand")
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.plot()







In [ ]:
import pandas as pd
full_data = pd.read_parquet("")
full_data

In [ ]:
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

def feature_engineering(data):
    data['date'] = pd.to_datetime(data['date'], format="%Y-%m-%d %H:%M:%S")

    data['hour'] = data['date'].dt.hour
    data['day_of_week'] = data['date'].dt.dayofweek
    data['month'] = data['date'].dt.month
    data['is_weekend'] = data['date'].isin([5,6]).astype(int)

    # add bank holiday
    holidays = calendar().holidays(start=data['date'].min(), end=data['date'].max())
    data['is_holiday'] = data['date'].isin(holidays).astype(int)
    return data

In [ ]:
full_data_with_FE = feature_engineering(full_data)
full_data_with_FE.to_parquet(TRANSFORMED_DATA_DIR/f'full_data_with_FE_parquet', index=False)

In [ ]:
import pandas as pd
from src.components.data_info import *
full_data_with_FE = pd.read_parquet(('..//data/transformed/full_data_with_FE.parquet'))
full_data_with_FE

In [ ]:
import tqdm
features, targets = transform_ts_data_into_features_and_target(full_data, input_seq_len=24*28*1, step_size=23)
print(f'{features.shape=}')
print(f'{targets.shape=}')

In [ ]:
features

In [ ]:
tabular_data = features
tabular_data['target_demand_next_hour'] = targets

from src.paths import TRANSFORMED_DATA_DIR
tabular_data.to_parquet(TRANSFORMED_DATA_DIR/'tabular_data')